In [45]:
import os
import re
import uuid
from docx import Document
from docx.shared import Pt
import win32com.client
from pymongo import MongoClient
import pyodbc
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

# ---------- הגדרות ----------
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["Manage_Files_DB"]
values_collection = mongo_db["Customers_Fields"]

# התחברות ל-SQL Server (התאימי לפרטים שלך)
def get_db_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-C0CP90K\\MSSQLSERVER01;"
        "DATABASE=ManageDB;"
        "Trusted_Connection=yes;"
    )

VARIABLE_REGEX = r"\{\{(.*?)\}\}"

# ---------- פונקציות ----------

def convert_doc_to_docx(input_path):
    try:
        word = win32com.client.Dispatch("Word.Application")
        doc = word.Documents.Open(input_path)
        output_path = os.path.splitext(input_path)[0] + ".docx"
        doc.SaveAs(output_path, FileFormat=16)  # 16 is the code for docx format
        doc.Close()
        word.Quit()
        return output_path
    except Exception as e:
        print("שגיאה בהמרה ל-docx:", e)
        return None

def create_highlighted_run(text, paragraph):
    run = paragraph.add_run(text)
    run.font.highlight_color = 7  # צהוב
    run.font.name = 'Arial'
    run.font.size = Pt(12)
    return run

def process_docx(path):
    doc = Document(path)
    conn = get_db_connection()
    cursor = conn.cursor()
    variables_not_found = []

    for para in doc.paragraphs:
        if not re.search(VARIABLE_REGEX, para.text):
            continue

        new_text = ""
        last_end = 0

        # עובר על כל המשתנים בתוך הטקסט
        for match in re.finditer(VARIABLE_REGEX, para.text):
            full_match = match.group(0)
            start, end = match.span()

            new_text += para.text[last_end:start]

            parts = match.group(1).split(',')
            if len(parts) < 5:
                new_text += full_match
                last_end = end
                continue

            var_name, var_type, var_len, var_required, var_description = [p.strip() for p in parts]

            # SQL: בדיקה אם קיים
            cursor.execute("SELECT COUNT(*) FROM Fields WHERE name = ?", (var_name,))
            if cursor.fetchone()[0] == 0:
                cursor.execute("INSERT INTO Fields (name, type, length, required) VALUES (?, ?, ?, ?)",
                               (var_name, var_type, int(var_len), int(var_required == '*')))
                conn.commit()

            # MongoDB
            value_doc = values_collection.find_one({"name": var_name})
            value = value_doc.get("value") if value_doc else None

            if not value:
                variables_not_found.append(var_name)
                value = "     "  # 5 רווחים

                # הוספת 5 רווחים מודגשים בצהוב
                create_highlighted_run("     ", para)

            new_text += str(value)
            last_end = end

        new_text += para.text[last_end:]
        para.clear()  # לא לשכוח לנקות את הפסקה לפני הוספת הטקסט החדש
        run = para.add_run(new_text)
        run.font.name = 'Arial'
        run.font.size = Pt(12)

    # שמירה עם שם + טופל
    dir_path = os.path.dirname(path)
    filename_wo_ext = os.path.splitext(os.path.basename(path))[0]
    output_path = os.path.join(dir_path, f"{filename_wo_ext} טופל.docx")
    doc.save(output_path)
    print("✅ קובץ נשמר כ:", output_path)
    if variables_not_found:
        print("⚠️ משתנים שלא נמצאו:", variables_not_found)

# ---------- הרצה ----------

# נתיב לקובץ Word לבדיקה (doc או docx)
input_path = "C:\\Users\\User\\Desktop\\מסמכים לפרויקט\\ממופה\\יפוי כוח.doc"  # שנה לפי הצורך

if input_path.endswith(".doc"):
    input_path = convert_doc_to_docx(input_path)

process_docx(input_path)


✅ קובץ נשמר כ: C:\Users\User\Desktop\מסמכים לפרויקט\ממופה\יפוי כוח טופל.docx
⚠️ משתנים שלא נמצאו: ['buyer_first_name', 'buyer_last_name', 'buyer_id', 'property_address_street', 'property_address_number', 'property_address_city', 'property_block_number', 'property_parcel_number', 'property_subparcel_numbers', 'seller_first_name', 'seller_last_name', 'seller_id']


In [60]:
import os
import re
import uuid
from docx import Document
from docx.shared import Pt
import win32com.client
from pymongo import MongoClient
import pyodbc
from docx.enum.text import WD_COLOR_INDEX

# ---------- הגדרות ----------
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["Manage_Files_DB"]
values_collection = mongo_db["Customers_Fields"]

# התחברות ל-SQL Server
def get_db_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-C0CP90K\\MSSQLSERVER01;"
        "DATABASE=ManageDB;"
        "Trusted_Connection=yes;"
    )

VARIABLE_REGEX = r"\{\{(.*?)\}\}"

# ---------- פונקציות ----------

def convert_doc_to_docx(input_path):
    try:
        word = win32com.client.Dispatch("Word.Application")
        doc = word.Documents.Open(input_path)
        output_path = os.path.splitext(input_path)[0] + ".docx"
        doc.SaveAs(output_path, FileFormat=16)  # 16 is docx format
        doc.Close()
        word.Quit()
        return output_path
    except Exception as e:
        print("שגיאה בהמרה ל-docx:", e)
        return None

def process_paragraphs(paragraphs, cursor, variables_not_found):
    for para in paragraphs:
        matches = list(re.finditer(VARIABLE_REGEX, para.text))
        if not matches:
            continue

        original_text = para.text
        para.clear()
        last_end = 0

        for match in matches:
            start, end = match.span()
            var_text = match.group(1)
            parts = var_text.split(',')

            before = original_text[last_end:start]
            if before:
                run = para.add_run(before)
                run.font.name = 'Arial'
                run.font.size = Pt(12)

            if len(parts) < 5:
                run = para.add_run(match.group(0))
                run.font.name = 'Arial'
                run.font.size = Pt(12)
                last_end = end
                continue

            var_name, var_type, var_len, var_required, var_description = [p.strip() for p in parts]

            # בדיקת קיום ב-SQL
            cursor.execute("SELECT COUNT(*) FROM Fields WHERE name = ?", (var_name,))
            if cursor.fetchone()[0] == 0:
                cursor.execute("INSERT INTO Fields (name, type, length, required) VALUES (?, ?, ?, ?)",
                               (var_name, var_type, int(var_len), int(var_required == '*')))
                cursor.connection.commit()

            # שליפת ערך מ-MongoDB
            value_doc = values_collection.find_one({"name": var_name})
            value = value_doc.get("value") if value_doc else None

            if value:
                run = para.add_run(str(value))
                run.font.name = 'Arial'
                run.font.size = Pt(12)
            else:
                variables_not_found.append(var_name)
                run = para.add_run("-----")
                run.font.highlight_color = WD_COLOR_INDEX.YELLOW
                run.font.name = 'Arial'
                run.font.size = Pt(12)

            last_end = end

        after = original_text[last_end:]
        if after:
            run = para.add_run(after)
            run.font.name = 'Arial'
            run.font.size = Pt(12)

def process_docx(path):
    doc = Document(path)
    conn = get_db_connection()
    cursor = conn.cursor()
    variables_not_found = []

    # פסקאות בגוף
    process_paragraphs(doc.paragraphs, cursor, variables_not_found)

    # טבלאות
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                process_paragraphs(cell.paragraphs, cursor, variables_not_found)

    # כותרות עליונות ותחתונות
    for section in doc.sections:
        process_paragraphs(section.header.paragraphs, cursor, variables_not_found)
        process_paragraphs(section.footer.paragraphs, cursor, variables_not_found)

    # שמירה
    dir_path = os.path.dirname(path)
    filename_wo_ext = os.path.splitext(os.path.basename(path))[0]
    output_path = os.path.join(dir_path, f"{filename_wo_ext} טופל.docx")
    doc.save(output_path)

    print("✅ קובץ נשמר כ:", output_path)
    if variables_not_found:
        print("⚠️ משתנים שלא נמצאו:", variables_not_found)

# ---------- הרצה ----------

# נתיב לקובץ Word לבדיקה (doc או docx)
input_path = "C:\\Users\\User\\Desktop\\מסמכים לפרויקט\\ממופה\\יפוי כוח.doc"

if input_path.endswith(".doc"):
    input_path = convert_doc_to_docx(input_path)

process_docx(input_path)


✅ קובץ נשמר כ: C:\Users\User\Desktop\מסמכים לפרויקט\ממופה\יפוי כוח טופל.docx
⚠️ משתנים שלא נמצאו: ['buyer_first_name', 'buyer_last_name', 'buyer_id', 'property_address_street', 'property_address_number', 'property_address_city', 'property_block_number', 'property_parcel_number', 'property_subparcel_numbers', 'seller_first_name', 'seller_last_name', 'seller_id']
